In [ ]:
tokens_df = pd.read_csv("../indexes.csv")
api = connect_to_twitter(global_token_index)

In [ ]:

start_with_tweet = "832883970295656320"
tweets_df = pd.read_csv('incivility-sage.csv')
tweets_df.id_str = tweets_df.id_str.apply(lambda x: str(int(x)))
tweets_df = tweets_df.loc[tweets_df[tweets_df['id_str']==start_with_tweet].index[0]+1:, :]
tweets_df.shape[0]

In [ ]:



max_token = 5
# token index
global_token_index = 0

# Auxilary Sleep for waiting to meet Twitter Developer API limitations
def aux_sleep():
    sleep_time = 15*600 # sleep for 15 minutes
    # show sleep progress
    for i in trange(sleep_time,
                    desc = f"sleep for {sleep_time/600} minutes"):
        time.sleep(0.1)


# Initial connection to Twitter
def connect_to_twitter(token_index = 0):

    consumer_key        = tokens_df.iloc[token_index]['consumer_key']
    consumer_secret     = tokens_df.iloc[token_index]['consumer_secret']
    access_token        = tokens_df.iloc[token_index]['access_token']
    access_token_secret = tokens_df.iloc[token_index]['access_token_secret']

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth,proxy="http://proxy-chain.intel.com:911")
    
    return api

#



In [ ]:


usersfile = open(f"user_tweets.csv","a+")
errorfile = open(f"error_tweets.csv","a+")

for tid in tqdm(tweets_df.id_str.unique()):

    for attempt in range(10):
        try:
            #print (f"tweet#: {tid}")
            res = api.get_status(tid)
            usersfile.write(f"{tid},{res.user.id}\n")

        except tweepy.errors.TweepyException as e:
            if 'Rate limit exceeded' in e.api_messages:
                global_token_index = (global_token_index + 1) % (max_token + 1)
                if global_token_index != 0:
                    print(f"Rate limit exceeded. Retrying with token #{global_token_index}")
                    api = connect_to_twitter(global_token_index)
                else:
                    print(f"Rate limit exceeded. Wait and retry {10-attempt} more times...")
                    time.sleep(1)
                    aux_sleep()
                continue #retry
            else:
                errorfile.write(f"{tid},{e.api_messages}\n")
                break # we failed not because of rate limit
        # We succeeded
        break
    #completed all attempts
